In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import random
import csv
from PIL import Image
import shutil
from tqdm import tqdm
import torch

from utils.utils0 import tensor_affine_transform, transform_to_displacement_field
from utils.utils1 import transform_points_DVF, ModelParams
from utils.SuperPoint import SuperPointFrontend, PointTracker
from utils.datagen import datagen

nn_thresh = 0.7
superpoint = SuperPointFrontend('utils/superpoint_v1.pth', nms_dist=4,
                          conf_thresh=0.015, nn_thresh=nn_thresh, cuda=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the data
data = pd.read_csv('Dataset/dataset_eye_actual_full.csv')


In [2]:
data.head()

,Unnamed: 0,source,Source ROI,target,Target ROI,training,Warped target images,Warped target ROI,Execution time,Directory
0,0,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lb
1,1,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lb
2,2,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lb
3,3,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lc
4,4,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lc


In [3]:
data.shape

(5040, 10)

In [4]:
# add columns 'keypoints', 'mse', 'tre'
data['keypoints'] = 0
data['mse'] = 0
data['tre'] = 0

# calculate the keypoints, mse, tre
for i in range(data.shape[0]):
    # keypoints = 0
    mse = 0
    tre = 0
    # read image file from column 'source' and 'target'
    source = plt.imread(data['source'][i])
    target = plt.imread(data['target'][i])
    # calculate MSE
    mse = np.mean((source - target) ** 2)
    # calculate TRE
    tre = np.sqrt(np.sum((source - target) ** 2))

    # save keypoints, mse, tre to the dataframe
    data['keypoints'][i] = 0
    data['mse'][i] = mse
    data['tre'][i] = tre

data.head()

/tmp/ipykernel_26277/2945763658.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['keypoints'][i] = 0
/tmp/ipykernel_26277/2945763658.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['mse'][i] = mse
/tmp/ipykernel_26277/2945763658.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '67.48326237996419' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data['mse'][i] = mse
/tmp/ipykernel_26277/2945763658.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,Unnamed: 0,source,Source ROI,target,Target ROI,training,Warped target images,Warped target ROI,Execution time,Directory,keypoints,mse,tre
0,0,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lb,0,67.483262,7284.984351
1,1,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lb,0,90.409568,8432.139527
2,2,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lb,0,90.971382,8458.298056
3,3,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lc,0,72.829871,7568.073797
4,4,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,Dataset/Dataset-processed/15-12-2559/2011248/L...,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lc,0,76.596774,7761.324243
